In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('100kRatings.csv')
df[:]

,userId,movieId,rating,timestamp
0,2071,296,2.0,909153539
1,2071,318,3.0,909153276
2,2071,337,1.0,909156342
3,2071,356,5.0,909152984
4,2071,377,4.0,909156750
...,...,...,...,...
133082,2987,176371,4.5,1525037133
133083,2987,177593,4.0,1525037138
133084,2987,177765,4.0,1525037136
133085,2987,182835,0.5,1525197138


In [3]:
mn = df['movieId'].max()
df['movieId'].min()
mn

207642

In [4]:
pop = np.zeros(mn + 1)
for i in df['movieId']:
    pop[i] += 1
popRank = np.argsort(pop)[::-1]
popTot = sum(pop)
popRank

array([   318,    356,    296, ..., 138000, 137999,      0], dtype=int64)

In [5]:
pop[popRank[:500]]

array([450., 445., 430., 414., 399., 393., 341., 337., 330., 324., 323.,
       318., 315., 310., 307., 303., 302., 296., 296., 289., 276., 273.,
       273., 267., 261., 259., 257., 244., 242., 240., 239., 236., 235.,
       232., 231., 230., 229., 225., 219., 219., 217., 214., 214., 213.,
       213., 212., 209., 208., 206., 205., 204., 204., 204., 200., 200.,
       195., 195., 193., 191., 189., 188., 188., 186., 185., 185., 185.,
       184., 184., 183., 182., 176., 176., 175., 175., 174., 173., 173.,
       171., 169., 169., 168., 168., 166., 166., 165., 164., 161., 161.,
       160., 159., 159., 159., 158., 158., 157., 156., 156., 155., 154.,
       154., 153., 153., 152., 151., 150., 150., 150., 150., 150., 149.,
       149., 149., 146., 146., 145., 145., 145., 144., 143., 143., 142.,
       142., 141., 141., 139., 139., 139., 138., 138., 136., 134., 134.,
       132., 132., 132., 132., 131., 131., 130., 130., 130., 129., 129.,
       128., 128., 127., 127., 127., 126., 125., 12

In [6]:
tmp = 0
sep = 0
while tmp < popTot * 0.75:
    tmp += pop[popRank[sep]]
    sep += 1
sep

1807

In [7]:
groups = df.groupby('userId').groups
un = len(groups)
un, groups

(917,
 {2071: Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
              ...
              116, 117, 118, 119, 120, 121, 122, 123, 124, 125],
             dtype='int64', length=126),
  2072: Int64Index([126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
              139, 140, 141, 142, 143, 144, 145],
             dtype='int64'),
  2073: Int64Index([146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
              159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
              172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184,
              185, 186, 187, 188],
             dtype='int64'),
  2074: Int64Index([189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
              202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214,
              215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227,
              228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240,
       

In [8]:
cnt = 0
for ind in groups:
    group = groups[ind]
    if len(group) < 20:
        cnt += 1
cnt

0

In [9]:
dic = {}
dic_inv = {}
for i in range(sep):
    dic[popRank[i]] = i
    dic_inv[i] = popRank[i]
0 in dic

False

In [10]:
i = 0
cnt = 0
data = []
for ind in groups:
    group = groups[ind]
    tp = []
    for j in group:
        movie = df['movieId'][j]
        if movie in dic:
            tp.append((movie, df['rating'][j] >= 3))
            cnt += 1
    data.append(tp)
cnt

99825

In [11]:
x = int(cnt * 0.8)
tp = 0
train = []
test = []
for i in data:
    if tp < x:
        train.append(i)
        for j in i:
            movie = j[0]
            value = j[1]
    else:
        test.append(i)
    tp += len(i)
len(train), len(test)

(719, 198)

In [12]:
mat = np.zeros([len(train), sep])
mat.shape

(719, 1807)

In [13]:
for i in range(len(train)):
    for j in train[i]:
        movie = j[0]
        value = j[1]
        mat[i][dic[movie]] = value

In [14]:
mat = mat.astype(int)
mat.shape

(719, 1807)

In [15]:
import csv
with open("mat.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(mat)
    f.close()

In [16]:
dic2 = {}
for i in dic:
    dic2[str(i)] = dic[i]

In [17]:
import pickle
f1 = open("dic", "wb")
pickle.dump(dic, f1)
f2 = open("dic_inv", "wb")
pickle.dump(dic_inv, f2)
f1.close()
f2.close()

In [18]:
f3 = open("test", "wb")
pickle.dump(test, f3)
f3.close()

In [19]:
test

[[(3, True),
  (50, True),
  (266, True),
  (296, True),
  (356, True),
  (380, True),
  (551, True),
  (778, True),
  (913, True),
  (924, True),
  (1136, True),
  (1197, True),
  (1203, True),
  (1233, True),
  (1266, True),
  (1270, True),
  (1275, False),
  (1282, True),
  (1380, True),
  (1394, True),
  (1597, False),
  (1619, True),
  (1673, True),
  (1732, True),
  (2000, True),
  (2072, False),
  (2288, True),
  (2291, True),
  (2395, True),
  (2505, False),
  (2571, True),
  (2572, True),
  (2617, False),
  (2640, True),
  (2890, True),
  (2948, True),
  (2968, True),
  (3168, True),
  (3259, True),
  (3298, True),
  (3435, True),
  (3498, True),
  (3671, True),
  (3948, True),
  (3994, True),
  (4011, True),
  (4085, True),
  (4128, False),
  (4226, True),
  (4720, True),
  (4848, True),
  (5010, True),
  (5049, True),
  (5952, True),
  (6188, True),
  (6287, False),
  (6711, True),
  (6953, True),
  (8528, False),
  (8984, True),
  (33679, True),
  (40819, True),
  (48385, T